### 框架干的最厉害的事就是帮我们把反向传播计算好了

In [52]:
import torch

需要求导可以这样定义数据

In [53]:
x = torch.rand(3,4,requires_grad=True)
print(x)

tensor([[0.3007, 0.4900, 0.2246, 0.3477],
        [0.7524, 0.2480, 0.1480, 0.8246],
        [0.0248, 0.2700, 0.7041, 0.8740]], requires_grad=True)


也可以这样

In [54]:
x = torch.rand(3,4)
x.requires_grad = True
print(x)

tensor([[0.9532, 0.0797, 0.1060, 0.2628],
        [0.7296, 0.8872, 0.0664, 0.5578],
        [0.9352, 0.7018, 0.2083, 0.1808]], requires_grad=True)


举个反向传播的例子

In [55]:
x = torch.rand(4,4)
w = x.new_ones(4,4,requires_grad=True)
b = torch.rand(4,4,requires_grad=True)
print(x)
print(w)
print(b)

tensor([[0.3653, 0.7470, 0.7855, 0.2712],
        [0.2295, 0.4453, 0.6024, 0.8549],
        [0.2226, 0.5510, 0.9166, 0.0029],
        [0.1206, 0.8039, 0.7707, 0.2976]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)
tensor([[0.7889, 0.5083, 0.4279, 0.6112],
        [0.1276, 0.7867, 0.1227, 0.1908],
        [0.8377, 0.3356, 0.0152, 0.2328],
        [0.8149, 0.9126, 0.9100, 0.0450]], requires_grad=True)


y没有设置requires_grad, 但计算w的梯度需要它，系统会自动设置

In [56]:
y = x * w
z = y + b
output = z.sum()
print(y)
print(z)
print(output)
print(x.requires_grad,w.requires_grad,y.requires_grad,b.requires_grad,z.requires_grad)
print(x.is_leaf,w.is_leaf,y.is_leaf,b.is_leaf,z.is_leaf)

tensor([[0.3653, 0.7470, 0.7855, 0.2712],
        [0.2295, 0.4453, 0.6024, 0.8549],
        [0.2226, 0.5510, 0.9166, 0.0029],
        [0.1206, 0.8039, 0.7707, 0.2976]], grad_fn=<MulBackward0>)
tensor([[1.1543, 1.2552, 1.2133, 0.8823],
        [0.3572, 1.2320, 0.7251, 1.0457],
        [1.0604, 0.8866, 0.9318, 0.2358],
        [0.9354, 1.7165, 1.6806, 0.3426]], grad_fn=<AddBackward0>)
tensor(15.6549, grad_fn=<SumBackward0>)
False True True True True
True True False True False


要对w求梯度，output先对z求导，得到的tensor为output对z的每一个元素求导(只能对叶子节点求导，刚才想对z求报错了)
然后乘z对y求导，z的每个元素由y和b相加而来，求导为全1矩阵
然后y对w求导，得到x矩阵
最终结果就是 全1矩阵 * 全1矩阵 * x矩阵 = x矩阵


In [60]:
output.backward(retain_graph=True)#retain_graph=True使反向传播后中间变量不被删除，可以多次执行，反向传播会将梯度叠加，记得清零
print(w.grad)
print(b.grad)

tensor([[1.4613, 2.9878, 3.1418, 1.0847],
        [0.9181, 1.7813, 2.4096, 3.4197],
        [0.8905, 2.2041, 3.6665, 0.0118],
        [0.4823, 3.2157, 3.0827, 1.1902]])
tensor([[4., 4., 4., 4.],
        [4., 4., 4., 4.],
        [4., 4., 4., 4.],
        [4., 4., 4., 4.]])
